## Hash Tables

### Phone Book (Open Addressing Method with Double Hashing and Rehashing)

**Task:** In this task your goal is to implement a simple phone book manager. It should be able to process the
following types of user’s queries:

- **add** number name. It means that the user adds a person with name **name** and phone number **number** to the phone book. If there exists a user with such number already, then your manager has to overwrite the corresponding name.
- **del** number. It means that the manager should erase a person with number **number** from the phone book. If there is no such person, then it should just ignore the query.
- **find** number. It means that the user looks for a person with phone number **number**. The manager should reply with the appropriate name, or with string “not found" (without quotes) if there is no such person in the book.

**Input Format:** There is a single integer $N$ in the first line — the number of queries. It’s followed by $N$
lines, each of them contains one query in the format described above.

**Constraints:** $1 \leq N \leq 10^5$. All phone numbers consist of decimal digits, they don’t have leading zeros, and
each of them has no more than $7$ digits. All names are non-empty strings of latin letters, and each of
them has length at most $15$. It’s guaranteed that there is no person with name “not found".

**Output Format:** Print the result of each **find** query — the name corresponding to the phone number or
“not found" (without quotes) if there is no person in the phone book with such phone number. Output
one result per line in the same order as the **find** queries are given in the input.

In [5]:
class HashTable():
    size = 13
    prime = 11
    max_load_factor = 0.75
    
    def __init__(self):
        self.num_of_keys = 0
        self.table = [None] * self.size
         
    def hashing(self, key):
        return key % self.size
    
    def double_hashing(self, key):
        #print('hash2: ', self.prime - key % self.prime)
        return self.prime - key % self.prime
    
    def insert(self, key, value):
        self.num_of_keys += 1
        hashed_key = self.hashing(key)
        #print('hash1: ', hashed_key)
        if self.table[hashed_key] == None:
            self.table[hashed_key] = (key, value)
            #print('table after insertion: ', self.table)
            self.rehash(self.table)
            return
        i = 1
        while self.table[hashed_key] != None and self.table[hashed_key] != -1:
            if self.table[hashed_key][0] == key:
                self.table[hashed_key] = (key, value)
                #print('table after insertion of same person: ', self.table)
                return
            hashed_key += i * self.double_hashing(key)
            hashed_key = hashed_key % self.size
            #print('double hash for insertion: ', hashed_key)
            i += 1 
        self.table[hashed_key] = (key, value)
        #print('table after insertion of different person: ', self.table)
        self.rehash(self.table)

    def find(self, key):
        hashed_key = self.hashing(key)
        i = 1
        while self.table[hashed_key] != None:
            if self.table[hashed_key] != -1 and self.table[hashed_key][0] == key:
                print(self.table[hashed_key][1])
                return
            hashed_key += i * self.double_hashing(key)
            hashed_key = hashed_key % self.size
            #print('double hash for find: ', hashed_key)
            i += 1
        print('not found')
        return
        
    def delete(self, key):
        hashed_key = self.hashing(key)
        i = 1
        while self.table[hashed_key] != None and self.table[hashed_key] != -1:
            if self.table[hashed_key][0] == key:
                self.num_of_keys -= 1
                self.table[hashed_key] = -1
                #print('table after deletion: ', self.table)
                return
            hashed_key += i * self.double_hashing(key)
            hashed_key = hashed_key % self.size
            #print('double hash for deletion: ', hashed_key)
            i += 1
            
    def rehash(self, table):
        load_factor = self.num_of_keys / self.size
        #print('load_factor', load_factor)
        if load_factor > self.max_load_factor:
            #print('rehashing!!!')
            self.num_of_keys = 0
            temp = self.table
            self.size *= 2
            self.table = [None] * self.size
            for i in temp:
                if i != None and i != -1:
                    self.insert(i[0], i[1])
                
    def print_table(self):
        print(self.table)
                
if __name__ == '__main__':
    phone_book = HashTable()
    n = int(input())
    query = list(input().split())
    i = 0
    while i < len(query):
        if query[i] == 'add':
            phone_book.insert(int(query[i+1]), query[i+2])
            i += 3
        elif query[i] == 'del':
            phone_book.delete(int(query[i+1]))
            i += 2
        elif query[i] == 'find':
            phone_book.find(int(query[i+1]))
            i += 2
        else:
            assert(0)

10
add 3610838 Ata add 3163318 Emi find 5463423 add 4995588 Men add 6727102 Ayturan find 3610838 del 3610838 find 4995588 find 3610838 find 6727102
not found
Ata
Men
not found
Ayturan


    Alternative solution by using Python dictionary.

In [4]:
if __name__ == '__main__':
    phone_book = {}
    n = int(input())
    for i in range(n):
        query = input().split()
        if query[0] == 'add':
            phone_book[query[1]] = query[2]
        elif query[0] == 'del':
            phone_book.pop(query[1], None)
        elif query[0] == 'find':
            if query[1] not in phone_book.keys():
                print('not found')
            else:
                print(phone_book[query[1]])
        else:
            assert(0)
    print(phone_book)

10
add 4995588 Arturk
add 7563690 Aqsin
add 8306699 Adil
find 4531022
not found
add 7777777 Noone
add 3131033 Orxan
find 7563690
Aqsin
del 7777777
add 4995588 Erturk
find 4995588
Erturk
{'4995588': 'Erturk', '7563690': 'Aqsin', '8306699': 'Adil', '3131033': 'Orxan'}


### Hashing with Chains (Seperate Chaining Method with Linked List)

**Task:** In this task your goal is to implement a hash table with lists chaining. You are already given the
number of buckets $m$ and the hash function. It is a polynomial hash function:
$$h(S) = \Bigg(\sum_{i=0}^{|S|-1} S[i] \space x^i \space mod \space p \Bigg) \space mod \space m$$

where $S[i]$ is the ASCII code of the $i$-th symbol of $S$, $p = 1 000 000 007$ and $x = 263$. Your program
should support the following kinds of queries:
- **add** string — insert string into the table. If there is already such string in the hash table, then just ignore the query.
- **del** string — remove string from the table. If there is no such string in the hash table, then just ignore the query.
- **find** string — output “yes" or “no" (without quotes) depending on whether the table contains string or not.
- **check** $i$ — output the content of the $i$-th list in the table. Use spaces to separate the elements of the list. If $i$-th list is empty, output a blank line.

When inserting a new string into a hash chain, you must insert it in the beginning of the chain.

**Input Format:** There is a single integer $m$ in the first line — the number of buckets you should have. The
next line contains the number of queries $N$. It’s followed by $N$ lines, each of them contains one query
in the format described above.

**Constraints:** $1 \leq N \leq 10^5; \frac{N}{5} \leq m \leq N$. All the strings consist of latin letters. Each of them is non-empty and has length at most $15$.

**Output Format:** Print the result of each of the **find** and **check** queries, one result per line, in the same
order as these queries are given in the input.

In [10]:
class Node:
    def __init__(self, key):
        self.key = key
        self.next = None

class LinkedList:
    def __init__(self):
        self.head = None
        self.tail = None
        
    def push_front(self, key):
        node = Node(key)
        node.next = self.head
        self.head = node
        if self.tail == None:
            self.tail = self.head
    
    def pop_front(self):
        if self.head == None:
            return
        else:
            self.head = self.head.next
        if self.head == None:
            self.tail = self.head
    
    def find_key(self, key):
        temp = self.head
        while(temp != None and temp.key != key):
            temp = temp.next
        if self.head == None or temp == None:
            print('no')
        else:
            print('yes')          

    def delete_key(self, key):
        temp = self.head
        if temp == None:
            return
        elif temp.key == key:
            self.pop_front()
        else:
            while(temp.next != None and temp.next.key != key):
                temp = temp.next
            if temp.next == None and temp.key != key:
                return
            else:
                if temp.next.next != None:
                    temp.next = temp.next.next
                else:
                    self.tail = temp
                    temp.next = None
    
    def print_linked_list(self):
        temp = self.head
        if temp == None:
            print()
        while(temp != None):
            if temp != self.tail:
                print(temp.key, end = ' ')
            else:
                print(temp.key)
            temp = temp.next
            
'''
# Alternative Hash Function:
def hash_func(self, string):
    hash_value = 0
    for s in reversed(string):
        hash_value = (hash_value * self.multiplier + ord(s)) % self.prime
    return hash_value % self.bucket_count
'''
        
class HashChainTable:
    multiplier = 263
    prime = 1000000007

    def __init__(self, bucket_count):
        self.bucket_count = bucket_count
        self.hash_table = [None] * self.bucket_count
        for i in range(self.bucket_count) :
            self.hash_table[i] = LinkedList()

    def hash_func(self, string):
        hash_value = 0
        for i in range(len(string)):
            hash_value = (hash_value + ord(string[i]) * (self.multiplier ** i)) % self.prime
        return hash_value % self.bucket_count
    
    def insert(self, string):
        index = self.hash_func(string)
        self.hash_table[index].push_front(string)

    def find(self, string):
        index = self.hash_func(string)
        self.hash_table[index].find_key(string)
        
    def delete(self, string):
        index = self.hash_func(string)
        self.hash_table[index].delete_key(string)
                
    def print_chain(self, index):
        self.hash_table[index].print_linked_list()

if __name__ == '__main__':
    bucket_count = int(input())
    chain = HashChainTable(bucket_count)
    query_count = int(input())
    elements = []
    query = list(input().split())
    i = 0
    while i < len(query):
        if query[i] == 'add':
            if query[i+1] not in elements:
                chain.insert(query[i+1])
                elements.append(query[i+1])
            i += 2
        elif query[i] == 'del':
            if query[i+1] in elements:
                chain.delete(query[i+1])
                elements.remove(query[i+1])
            i += 2
        elif query[i] == 'find':
            chain.find(query[i+1])
            i += 2
        elif query[i] == 'check':
            chain.print_chain(int(query[i+1]))
            i += 2
        else:
            assert(0)

5
12
add world add HellO check 4 find World find world del world check 4 del HellO add luck add GooD check 2 del good
HellO world
no
yes
HellO
GooD luck


    Alternative solution by using Python dictionary and deque() function.

In [17]:
from collections import deque

def hash_func(s, m):
    h = 0
    for i in range(len(s)):
        h = (h + ord(s[i]) * (263 ** i)) % 1000000007
    return h % m

if __name__ == '__main__':
    hash_chains = {}
    elements = []
    m = int(input())
    for i in range(m):
        hash_chains[i] = deque()   
    n = int(input())
    for _ in range(n):
        query = input().split()
        if query[0] == 'add':
            if query[1] not in elements:
                elements.append(query[1])
                index = hash_func(query[1], m)
                hash_chains[index].appendleft(query[1])
        elif query[0] == 'del':
            if query[1] in elements:
                elements.remove(query[1])
                index = hash_func(query[1], m)
                hash_chains[index].remove(query[1])
        elif query[0] == 'find':
            index = hash_func(query[1], m)
            if query[1] in hash_chains[index]:
                print('yes')
            else:
                print('no')
        elif query[0] == 'check':
            if len(hash_chains.get(int(query[1]))) == 0:
                print()
            for i in range(len(hash_chains.get(int(query[1])))):
                if i != len(hash_chains.get(int(query[1]))) - 1:
                    print(hash_chains.get(int(query[1]))[i], end = ' ')
                else:
                    print(hash_chains.get(int(query[1]))[i])
        else:
            assert(0)
    print(hash_chains)

3
12
check 0

find help
no
add help
add del
add add
find add
yes
find del
yes
del del
find del
no
check 0

check 2

check 1
add help
{0: deque([]), 1: deque(['add', 'help']), 2: deque([])}


    Alternative solution by using Python dictionary and deque() function - Fast solution.

In [1]:
from collections import deque

def hash_func(string):
    hash_value = 0
    for i in range(len(string)):
        hash_value = (hash_value + ord(string[i]) * (multiplier ** i)) % prime
    return hash_value % bucket_count

def addition(string):
    hash_value = hash_func(string)
    if string not in bucket[hash_value]:
        bucket[hash_value].appendleft(string)
        
def deletion(string):
    hash_value = hash_func(string)
    if string in bucket[hash_value]:
        bucket[hash_value].remove(string)
    
def search(string):
    hash_value = hash_func(string)
    print("yes" if string in bucket[hash_value] else "no")
    
def print_bucket(index):
    if index not in bucket:
        print()
    else:
        print(' '.join(bucket[index]))

if __name__ == '__main__':
    global multiplier, prime, bucket_count, bucket
    multiplier = 263
    prime = 1000000007
    
    bucket_count = int(input())
    bucket = {}
    for i in range(bucket_count):
        bucket[i] = deque()
    
    query_count = int(input())
    queries = [input() for _ in range(query_count)]
        
    for query in queries:
        operation, argument = query.split()
        if operation == 'add':
            addition(argument)
        if operation == 'del':
            deletion(argument)
        if operation == 'find':
            search(argument)
        if operation == 'check':
            print_bucket(int(argument))

4
8
add test
add test
find test
del test
find test
find Test
add Test
find Test
yes
no
no
yes


### Find Pattern in Text (Rabin–Karp’s Algorithm)

**Task:** In this problem your goal is to implement the Rabin–Karp’s algorithm for searching the given pattern
in the given text.

**Input Format:** There are two strings in the input: the pattern $P$ and the text $T$.

**Constraints:** $1 \leq |P| \leq |T| \leq 5 \cdot 10^5$.
The total length of all occurrences of $P$ in $T$ doesn’t exceed 10^8.
The pattern and the text contain only latin letters.

**Output Format:** Print all the positions of the occurrences of $P$ in $T$ in the ascending order. Use 0-based
indexing of positions in the the text $T$.

**Polynomial Hash Function:**

$H[i+1] = \sum_{j=i+1}^{i+|P|} T[j] \space x^{j-i-1} \space mod \space p$

$H[i] = \sum_{j=i}^{i+|P|-1} T[j] \space x^{j-i} \space mod \space p$

After recurrence of hashes, $H[i] = xH[i+1] + (T[i] - T[i + |P|]x^{|P|}) \space mod \space p$

In [1]:
from random import randint

def find_pattern_naive(text, pattern):
    result = []
    for i in range(len(text) - len(pattern) + 1):
        if are_equal(text[i: i + len(pattern)], pattern):
            result.append(i)
    return result

def are_equal(first, second):
    if len(first) != len(second):
        return False
    for i in range(len(first)):
        if first[i] != second[i]:
            return False
    return True

def poly_hash(string, p, x):
    hash_value = 0
    for i in range(len(string)):
        hash_value = (hash_value + ord(string[i]) * (x ** i)) % p
    return hash_value    

def precompute_hashes(text, pattern_length, p, x):
    Hash = [None] * (len(text) - pattern_length + 1)
    last_substring = text[len(text) - pattern_length:]
    Hash[len(text) - pattern_length] = poly_hash(last_substring, p, x)
    for i in range(len(text) - pattern_length - 1, -1 , -1):
        last = i + pattern_length
        Hash[i] = (x * Hash[i + 1] + ord(text[i]) - (ord(text[last]) * (x ** pattern_length) % p)) % p
    return Hash

def Rabin_Karp(text, pattern):
    p = 31
    x = randint(0, p - 1)
    result = []
    hash_pattern = poly_hash(pattern, p, x)
    pattern_length = len(pattern)
    Hash = precompute_hashes(text, pattern_length, p, x)
    for i in range(len(text) - len(pattern) + 1):
        if hash_pattern != Hash[i]:
            continue
        if are_equal(text[i: i + len(pattern)], pattern):
            result.append(i)
    return result

if __name__ == '__main__':
    pattern = input()
    text = input()
    print(*Rabin_Karp(text, pattern))

ko
adrenokortikotropin
6 11


    Alternative solution by using Python hash() function.

In [12]:
def find_pattern(text, pattern):
    hash_pattern = hash(pattern)
    hashes = []
    for i in range(len(text) - len(pattern) + 1):
        hashes.append(hash(text[i: i + len(pattern)]))
    result = []
    for h in range(len(hashes)):
        if hashes[h] == hash_pattern:
            result.append(h)
    return result
        
if __name__ == '__main__':
    pattern = input()
    text = input()
    print(*find_pattern(text, pattern))

alma
almaniyada almadan qalma
0 11 20


### Substring Equality

**Task:** In this problem, you will use hashing to design an algorithm that is able to preprocess a given string $s$
to answer any query of the form “are these two substrings of $s$ equal?” efficiently. This, in turn, is a basic
building block in many string processing algorithms.

**Input Format:** The first line contains a string $s$ consisting of small Latin letters. The second line contains
the number of queries $q$. Each of the next $q$ lines specifies a query by three integers $a$, $b$, and $l$.

**Constraints:** $1 \leq |s| \leq 500000; 1 \leq q \leq 100000; 0 \leq a, b \leq |s| − l$ (hence the indices $a$ and $b$ are 0-based).

**Output Format:** For each query, output **“Yes”** if $s_a s_{a+1} \dotsc s_{a+l-1} = s_b s_{b+1} \dotsc s_{b+l-1}$ are equal, and **“No”** otherwise.

**Polynomial Hash Function:**

$H(t) = \sum_{j=0}^{m-1} t_j x^{m-j-1}= t_0 x^{m-1} + t_1 x^{m-2} + \dots + t_{m-2} x + t_{m-1}$ 

$let \space h[0] = 0, \space and \space for \space 1 \leq i \leq n, \space let \space h[i] = H(s_0 s_1 \dotsc s_{i-1})$

$H(s_a s_{a+1} \dotsc s_{a+l-1}) = h[a+l] - x^l h[a]$

In [3]:
class Solver:
    def __init__(self, s):
        self.s = s
    def ask(self, a, b, l):
        return s[a:a+l] == s[b:b+l]
    
from random import randint

def precompute_hashes(text, l, m, x):
    hash_array = [None] * (len(text) + 1)
    hash_array[0] = 0
    for i in range(1, len(text) + 1):
        hash_array[i] = (x * hash_array[i - 1] + ord(text[i - 1])) % m
        
    Hash = [None] * (len(text) - l + 1)
    for i in range(0, len(text) - l + 1):
        Hash[i] = (hash_array[i + l] - (hash_array[i] * (x ** l) % m)) % m
    return Hash

def substring_equality(text, a, b, l):
    m_1 = 10 ** 9 + 7
    m_2 = 10 ** 9 + 9
    x = randint(1, 10 ** 9)
    
    Hash_1 = precompute_hashes(text, l, m_1, x)
    Hash_2 = precompute_hashes(text, l, m_2, x)
    
    if Hash_1[a] == Hash_1[b] and Hash_2[a] == Hash_2[b]:
        return True
    return False

if __name__ == '__main__':
    text = input()
    query = int(input())
    # naive_solver = Solver()
    for i in range(query):
        a, b, l = list(map(int, input().split()))
        # print("Yes" if naive_solver.ask(a, b, l) else "No")
        print("Yes" if substring_equality(text, a, b, l) else "No")

temerkuzlesme
3
3 6 3
No
2 11 2
Yes
0 7 5
No


    Alternative solution by adding Python dictionary.

In [2]:
from random import randint

def precompute_hashes(text, l, m, x):
    hash_array = [None] * (len(text) + 1)
    hash_array[0] = 0
    for i in range(1, len(text) + 1):
        hash_array[i] = (x * hash_array[i - 1] + ord(text[i - 1])) % m
        
    Hash = [None] * (len(text) - l + 1)
    for i in range(0, len(text) - l + 1):
        Hash[i] = (hash_array[i + l] - (hash_array[i] * (x ** l) % m)) % m
    
    HD[(l, m)] = Hash

def substring_equality(text, a, b, l):
    m_1 = 10 ** 9 + 7
    m_2 = 10 ** 9 + 9
    x = randint(1, 10 ** 9)
    
    if (l, m_1) not in HD:
        precompute_hashes(text, l, m_1, x)
    if (l, m_2) not in HD:
        precompute_hashes(text, l, m_2, x)
    
    if HD[(l, m_1)][a] == HD[(l, m_1)][b] and HD[(l, m_2)][a] == HD[(l, m_2)][b]:
        return True
    return False

if __name__ == '__main__':
    text = input()
    query = int(input())
    global HD
    HD = {}
    for i in range(query):
        a, b, l = list(map(int, input().split()))
        print("Yes" if substring_equality(text, a, b, l) else "No")

trololo
4
0 0 7
Yes
2 4 3
Yes
3 5 1
Yes
1 3 2
No


    Alternative solution by using Python hash() function.

In [15]:
def precompute_hashes(text, l):
    Hash = []
    for i in range(len(text) - l + 1):
        Hash.append(hash(text[i: i + l]))
    HD[l] = Hash

def substring_equality(text, a, b, l):
    if l not in HD:
        precompute_hashes(text, l)
    if HD[l][a] == HD[l][b]:
        return True
    else:
        return False

if __name__ == '__main__':
    text = input()
    query = int(input())
    global HD
    HD = {}
    for i in range(query):
        a, b, l = list(map(int, input().split()))
        print("Yes" if substring_equality(text, a, b, l) else "No")

dandanakan
5
0 3 3
Yes
0 1 6
No
4 5 3
No
1 8 2
Yes
0 5 2
No


### Longest Common Substring

**Task:** In the longest common substring problem one is given two strings $s$ and $t$ and the goal is to find a string $w$
of maximal length that is a substring of both $s$ and $t$. This is a natural measure of similarity between two
strings. The problem has applications in text comparison and compression as well as in bioinformatics.

The problem can be seen as a special case of the edit distance problem (where only insertions and
deletions are allowed). Hence, it can be solved in time $O(|s| · |t|)$ using dynamic programming. Later in
this specialization, we will learn highly non-trivial data structures for solving this problem in linear time
$O(|s| + |t|)$. In this problem, your goal is to use hashing to solve it in almost linear time.

**Input Format:** Every line of the input contains two strings $s$ and $t$ consisting of lower case Latin letters.

**Constraints:** The total length of all $s$’s as well as the total length of all $t$’s does not exceed $100000$.

**Output Format:** For each pair of strings $𝑠$ and $t_i$ find its longest common substring and specify it by
outputting three integers: its starting position in $s$, its starting position in $t$ (both 0-based), and its
    length. More formally, output integers $0 \leq i < |s|, 0 \leq j < |t|$, and $𝑙 \geq 0$ such that $s_i s_{i+1} \dotsc s_{i+l-1} = t_j t_{j+1} \dotsc t_{j+l-1}$ and $l$ is maximal. (As usual, if there are many such triples with maximal $l$, output any
of them.)

In [2]:
from collections import namedtuple
import random

Answer = namedtuple('answer_type', 'i j len')

def naive_solver(s, t):
    ans = Answer(0, 0, 0)
    for i in range(len(s)):
        for j in range(len(t)):
            for l in range(min(len(s) - i, len(t) - j) + 1):
                if (l > ans.len) and (s[i:i+l] == t[j:j+l]):
                    ans = Answer(i, j, l)
    return ans

def precompute_hashes(text, l, p, x):
    hash_array = [None] * (len(text) + 1)
    hash_array[0] = 0
    for i in range(1, len(text) + 1):
        hash_array[i] = (x * hash_array[i - 1] + ord(text[i - 1])) % p
        
    Hash = [None] * (len(text) - l + 1)
    for i in range(0, len(text) - l + 1):
        Hash[i] = (hash_array[i + l] - (hash_array[i] * (x ** l) % p)) % p
    return Hash    

def longest_common_substring(string_1, string_2):
    p_1 = 10 ** 9 + 39
    p_2 = 10 ** 9 + 61
    x = random.randint(1, 10 ** 9)
    
    if len(string_1) <= len(string_2):
        pattern = string_1
        text = string_2
    else:
        pattern = string_2
        text = string_1
    
    left = 0
    right = len(pattern)
    
    first_index = 0
    second_index = 0

    while left <= right and left <= len(pattern) and right >= 0:
        breakFlag = False
        tempFlag = False
        
        mid = (left + right) // 2

        Hash_small_1 = precompute_hashes(pattern, mid, p_1, x)
        Hash_small_2 = precompute_hashes(pattern, mid, p_2, x)
    
        Hash_big_1 = precompute_hashes(text, mid, p_1, x)
        Hash_big_2 = precompute_hashes(text, mid, p_2, x)
 
        for i in range(0, len(Hash_small_1)):
            for j in range(0, len(Hash_big_1)):
                if Hash_small_1[i] == Hash_big_1[j] and Hash_small_2[i] == Hash_big_2[j]:
                    if left != mid:
                        first_index = i
                        second_index = j
                        left = mid
                        breakFlag = True
                        break
                    else:
                        first_index = i
                        second_index = j
                        tempFlag = True
                        left += 1
                        break
            if breakFlag == True or tempFlag == True:
                break
                
        if breakFlag == True or tempFlag == True:
            continue
        elif left == right:
            mid -= 1
            break
        else:
            right = mid
    
    if len(string_1) <= len(string_2):
        return first_index, second_index, mid
    else:
        return second_index, first_index, mid

if __name__ == '__main__':
    for _ in range(10):
        string_1, string_2 = input().split()
        # naive_solver = naive_solver(string_1, string_2)
        # print(ans.i, ans.j, ans.len)
        print(*longest_common_substring(string_1, string_2))

voteforthegreatazerbaijanforyou choosethegreatazerbaijanianfuture
7 6 18
meselcun teselli
1 1 4
qazaxistan qirgizistan
5 6 5
medeniyyetsiz mahiyyetdenkenar
5 3 5
baababbbbabaababaa babbabaababaabbaabaa
7 2 11
babababaaaaababaab bbbabbaababb
10 6 5
baaababbbbaa aababaabaaaabbaabaa
2 0 5
bbbabbbababbba baaabbaaabb
3 3 3
abaaabaabbaa aaabbbaaaabb
1 5 4
babbbabbabb abbaabaaab
5 0 4


### Pattern Matching with Mismatches

**Task:** A natural generalization of the pattern matching problem is the following: find all text locations where distance
from pattern is sufficiently small. This problems has applications in text searching (where mismatches
correspond to typos) and bioinformatics (where mismatches correspond to mutations).

For an integer parameter $k$ and two strings $t = t_o t_1 \dotsc t_{m-1}$ and $p = p_o p_1 \dotsc p_{n-1}$, we say that
$p$ occurs in $t$ at position $i$ with at most $k$ mismatches if the strings $p$ and $t[i:i+p]=t_i t_{i+1} \dotsc t_{i+n-1}$ differ in at most $k$ positions.

**Input Format:** Every line of the input contains an integer $k$ and two strings $t$ and $p$ consisting of lower
case Latin letters.

**Constraints:** $0 \leq k \leq 5, 1 \leq |t| \leq 200000, 1 \leq |p| \leq min(|t|, 100000)$. The total length of all $t$’s does not exceed $200000$, the total length of all $p$’s does not exceed $100000$.

**Output Format:** For each triple $(k, t, p)$, find all positions $0 \leq i_1 < i_2 < \dots < i_l < |t|$ where $p$ occurs in $t$ with at most $k$ mismatches. Output $l$ and $i_1, i_2, \dotsc , i_l$.

In [2]:
def compute_mismatch(k, text, pattern, i, left, right):  
    global mismatch
    if mismatch > k:
        return 
    if right >= left:
        mid = (right + left) // 2   
        if text[i + mid] != pattern[mid]:
            mismatch += 1
        if len(pattern[:mid]) != 0:
            if text[i:i + mid] != pattern[:mid]:
                compute_mismatch(k, text, pattern, i, left, mid - 1)           
        if len(pattern[mid + 1:]) != 0:
            if text[i + mid + 1:] != pattern[mid + 1:]:
                compute_mismatch(k, text, pattern, i, mid + 1, right)          
    else:
        return
            
def matching_with_mismatches(k, text, pattern):
    global mismatch
    result = []
    for i in range(len(text) - len(pattern) + 1):
        mismatch = 0
        left = 0
        right = len(pattern) - 1
        compute_mismatch(k, text, pattern, i, left, right)
        if mismatch <= k:
            result.append(i)  
    return result

if __name__ == '__main__':
    for _ in range(7):
        k, text, pattern = input().split()
        result = matching_with_mismatches(int(k), text, pattern)
        print(len(result), *result)

0 ababab baaa
0
1 ababab baaa
1 1
1 xabcabc ccc
0
2 xabcabc ccc
4 1 2 3 4
3 aaa xxx
1 0
1 oojjjjojjoojjjjjojjjoooojjj oojjjjojo
1 0
1 catchmeifyoucan can
2 0 12
